# Temperature Analysis I

In [1]:
#import dependencies
import pandas as pd
from datetime import datetime as dt
from scipy import stats

In [2]:
#import csv
measurements_df = pd.read_csv('Resources/hawaii_measurements.csv')
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [3]:
#convert date column to datetime
for index, row in measurements_df.iterrows():
    row['date'] = dt.strptime(row['date'], '%Y-%m-%d')

In [4]:
#set index to date column
measurements_df.set_index('date', inplace=True)
measurements_df.head()

,station,prcp,tobs
date,,,
2010-01-01,USC00519397,0.08,65
2010-01-02,USC00519397,0.00,63
2010-01-03,USC00519397,0.00,74
2010-01-04,USC00519397,0.00,76
2010-01-06,USC00519397,NaN,73


# June and December Data Comparison

In [26]:
#dictionaries to hold rows
june_dict = {}
dec_dict = {}

#pulling needed rows
for index, row in measurements_df.iterrows():
    dt_index = dt.strptime(index, '%Y-%m-%d')
    if dt_index.strftime('%m') == '06':
        june_dict[index] = row
    elif dt_index.strftime('%m') == '12':
        dec_dict[index] = row

In [29]:
#june data frame
june_df = pd.DataFrame.from_dict(june_dict, orient='index')
june_df.head()

,station,prcp,tobs
2010-06-01,USC00516128,0.08,70
2010-06-02,USC00516128,0.00,78
2010-06-03,USC00516128,0.10,73
2010-06-04,USC00516128,0.01,68
2010-06-05,USC00516128,0.09,79


In [30]:
#december data frame
dec_df = pd.DataFrame.from_dict(dec_dict, orient='index')
dec_df.head()

,station,prcp,tobs
2010-12-01,USC00516128,1.48,71
2010-12-03,USC00516128,1.59,67
2010-12-04,USC00516128,0.19,77
2010-12-06,USC00516128,0.00,61
2010-12-07,USC00516128,0.00,62
